In [87]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [88]:
# partly from https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [89]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [90]:
RUG = pd.read_pickle("../obfuscated_data.pkl")
RUG.interpolate(method='linear', inplace=True)#, limit=20)
RUG = RUG[::10]
dfs = [RUG.filter([i]) for i in RUG]

In [91]:
dfs = [i.dropna() for i in dfs]

In [92]:
# RUG

# Timeseries LSTM

In [98]:
def func(df, name):

    df2 = df.copy()

    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(df2)

    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=3):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)
    
    # reshape into X=t and Y=t+1
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.001, verbose=2)

    # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.fit(trainX, trainY, epochs=100, verbose=2, callbacks=[early_stopping, reduce_lr], batch_size=256)

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    rmse_train = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    # print('Train Score: %.2f RMSE' % (trainScore))
    rmse_test = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    # print('Test Score: %.2f RMSE' % (testScore))

    mae_train = tf.keras.metrics.mean_absolute_error(trainY[0], trainPredict[:,0]).numpy()
    mae_test = tf.keras.metrics.mean_absolute_error(testY[0], testPredict[:,0]).numpy()

    mape_train = tf.keras.metrics.mean_absolute_percentage_error(trainY[0], trainPredict[:,0]).numpy()
    mape_test = tf.keras.metrics.mean_absolute_percentage_error(testY[0], testPredict[:,0]).numpy()

    return (name, (rmse_train, rmse_test, mae_train, mae_test, mape_train, mape_test))

In [99]:
results = []

for name, df in zip([i for i in RUG], dfs):
    print(name)
    
    if df.isnull().values.any():
        print("NaN")
        results.append((name, (np.nan, np.nan)))
        print("-------------")
        continue

    r = func(df, name)
    print("-------------")
    results.append(r)

Location 1 - flow
Epoch 1/100
803/803 - 9s - loss: 0.0205 - mse: 0.0205 - lr: 0.0010 - 9s/epoch - 11ms/step
Epoch 2/100
803/803 - 6s - loss: 0.0051 - mse: 0.0051 - lr: 0.0010 - 6s/epoch - 7ms/step
Epoch 3/100
803/803 - 6s - loss: 0.0043 - mse: 0.0043 - lr: 0.0010 - 6s/epoch - 8ms/step
Epoch 4/100
803/803 - 6s - loss: 0.0037 - mse: 0.0037 - lr: 0.0010 - 6s/epoch - 8ms/step
Epoch 5/100
803/803 - 6s - loss: 0.0033 - mse: 0.0033 - lr: 0.0010 - 6s/epoch - 7ms/step
Epoch 6/100


KeyboardInterrupt: 

In [16]:
# for i in results:
#     print(i)

In [23]:
lstm_results = [[i[0], i[1][0], i[1][1], i[1][2], i[1][3], i[1][4], i[1][5]] for i in results]

lstm_results = pd.DataFrame(lstm_results)

lstm_results.columns = ["Location", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", "Train MAPE", "Test MAPE"]
lstm_results.set_index("Location", inplace=True)
lstm_results.sort_index(inplace=True)
lstm_results = lstm_results.astype(float).round(3)

display(lstm_results)

# print(lstm_results.round(3).to_latex())

,Train RMSE,Test RMSE,Train MAE,Test MAE,Train MAPE,Test MAPE
Location,,,,,,
Location 1 - flow,26.171,26.418,13.970,17.016,6.925356e+07,6.263240e+02
Location 10 - flow,17.747,21.030,5.777,6.781,4.912472e+03,2.223822e+03
Location 11 - flow,5.360,2.073,1.298,0.610,5.014563e+08,5.259237e+08
Location 11 - head,25.438,0.411,0.408,0.254,5.280000e-01,4.720000e-01
Location 12 - head,0.687,0.578,0.298,0.273,4.530000e-01,4.120000e-01
Location 2 - consumption,0.284,0.406,0.098,0.238,2.172000e+01,9.884582e+04
Location 3 - consumption,0.409,0.472,0.136,0.207,6.514070e+06,6.530083e+06
Location 4 - consumption,0.354,0.688,0.079,0.306,2.295000e+00,2.198144e+04
Location 5 - consumption,0.207,0.405,0.078,0.280,5.142208e+04,1.809159e+06


In [24]:
cols = pd.MultiIndex.from_product([['Train', 'Test'], ['RMSE', 'MAE', 'MAPE']])
lstm_results.columns = cols
print(lstm_results.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Train} & \multicolumn{3}{l}{Test} \\
{} &    RMSE &     MAE &    MAPE &    RMSE &           MAE &          MAPE \\
Location                 &         &         &         &         &               &               \\
\midrule
Location 1 - flow        &  26.171 &  26.418 &  13.970 &  17.016 &  6.925356e+07 &  6.263240e+02 \\
Location 10 - flow       &  17.747 &  21.030 &   5.777 &   6.781 &  4.912472e+03 &  2.223822e+03 \\
Location 11 - flow       &   5.360 &   2.073 &   1.298 &   0.610 &  5.014563e+08 &  5.259237e+08 \\
Location 11 - head       &  25.438 &   0.411 &   0.408 &   0.254 &  5.280000e-01 &  4.720000e-01 \\
Location 12 - head       &   0.687 &   0.578 &   0.298 &   0.273 &  4.530000e-01 &  4.120000e-01 \\
Location 2 - consumption &   0.284 &   0.406 &   0.098 &   0.238 &  2.172000e+01 &  9.884582e+04 \\
Location 3 - consumption &   0.409 &   0.472 &   0.136 &   0.207 &  6.514070e+06 &  6.530083e+06 \\
Location 4 - consu

C:\Users\Martin\AppData\Local\Temp\ipykernel_15448\2898845625.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(lstm_results.to_latex())
